# Projeto Integração I

# Importação de Bibliotecas

In [25]:
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
import shapely.wkt
import pyproj
from shapely.geometry import Point

# Importação do Data Frame

Identificamos que os dados a serem lidos estão em um arquivo .csv e usamos o método read_csv do pandas para passar os dados em um formato de DataFrame

In [26]:
data_location = "../Data/Real_State_Properati.csv"
df_prop = pd.read_csv(data_location, sep = '\t')


FileNotFoundError: [Errno 2] No such file or directory: '../Data/Real_State_Properati.csv'

# Encontramos os tipos de cada coluna
Identificamos os tipos de cada coluna e printamos alguns dados para melhor entendimento dos dados

In [23]:
print(df_prop.dtypes)
df_prop.head()

NameError: name 'df_prop' is not defined

# Verificando quantidade de linhas do arquivo

In [ ]:
df_prop.shape[0]

# Modificando nome da coluna surface_total_in_m2 para total_m2
Facilitará quando precisarmos analisar e usar esse campo

In [ ]:
df_prop.rename(columns={'surface_total_in_m2': 'total_m2'}, inplace = True)
df_prop.columns

# Verificação de valores ausentes
Verificamos os valores ausentes em cada coluna. Conclusão: Não existem valores ausentes em nenhuma das colunas

In [ ]:
data_null = df_prop.apply(lambda x : x.isnull().sum())
data_null

# Verificação de valores menores ou iguais a zero nas colunas price e total_m2
Vamos verificar se existem valores menores ou iguais a zero nas colunas price e surface_total_in_m2. Sobre a coluna price não encontramos nenhum registro com valor menor ou igual a zero, mas sobre a coluna total_m2 encontramos 6 registros.

In [ ]:
data_menor_igual_zero = pd.DataFrame(df_prop, columns=['price','total_m2'])
data_menor_igual_zero = data_menor_igual_zero.apply(lambda x : x<=0).sum()
data_menor_igual_zero

# Verificando quais registros de total_m2 estão com valor menor ou igual a zero
Temos 6 registros com valor em total_m2 menor ou igual a zero. Nas próximas linhas veremos qual ação tomar diante desse fato.

In [ ]:
mask_m2_menor_igual_zero = df_prop['total_m2']<=0
data_m2_menor_igual_zero = df_prop.loc[mask_m2_menor_igual_zero]
data_m2_menor_igual_zero

# Vamos preencher o total_m2 usando a média por property_type e place_name
Primeiro iremos criar um novo dataframe apenas com os registros cujo total_m2 seja maior que zero. Depois disso aplicaremos a média de total_m2 por property_type e place_name e preencheremos os valores faltantes na coluna total_m2 com a média respectiva por property_name e place_name

In [ ]:
mask_m2_maior_zero = df_prop['total_m2']>0
data_m2_maior_zero = df_prop.loc[mask_m2_maior_zero]
data_m2_maior_zero = pd.DataFrame(data_m2_maior_zero, columns=['property_type','place_name','total_m2'])
data_m2_maior_zero

# Agrupando as médias de total_m2 por property_type e place_name

In [ ]:
data_media_m2_pp = data_m2_maior_zero.groupby(['property_type','place_name']).mean().reset_index()
data_media_m2_pp

# Atribuindo as médias de total_m2 agrupadas nas respectivas linhas respeitando property_type e place_name
Usando laço for para atualizar a média do total_m2 por property_type e place_name e depois realizando um check para confirmar que não exista nenhum registro com o total_m2 menor ou igual a zero.

In [ ]:
for i in df_prop[mask_m2_menor_igual_zero].index:
    property_type = df_prop.loc[i, 'property_type']
    place_name    = df_prop.loc[i, 'place_name']
    total_m2      = float(data_media_m2_pp[(data_media_m2_pp['property_type']==property_type) & (data_media_m2_pp['place_name']==place_name)]['total_m2'])
    df_prop.loc[i, 'total_m2'] = total_m2

# Verificando se sobrou algum registro com total_m2 menor ou igual a zero

In [ ]:
mask_m2_menor_igual_zero_check = df_prop['total_m2']<=0
data_m2_menor_igual_zero_check = df_prop.loc[mask_m2_menor_igual_zero_check]
data_m2_menor_igual_zero_check

# Verificando número de registrospor property_type

In [ ]:
df_prop.value_counts('property_type')

# Verificando número de registros por currency
Percebemos que existem registros em dólares e em pesos argentinos. Teremos que converter tudo a uma moeda só

In [ ]:
df_prop.value_counts('currency')

# Criando coluna para unificação de preços uma vez que temos preços em Pesos e Dólares. Cotação do Dólar em 118.44 22/05/2022

In [ ]:
df_prop['price_usd'] = df_prop.apply(lambda x: x['price']/1 if x['currency']=='USD' else x['price']/118.44, axis=1)
df_prop.head(10)

In [ ]:
# substituindo todas as moedas pesos pos dolar em currency
df_prop = df_prop.replace({'currency': 'ARS'}, 'USD')
df_prop.value_counts('currency')

# Verificando quantos registros existem por place_name

In [ ]:
df_prop.value_counts('place_name')

# Criação de uma coluna price_m2 que armazenará o preço em dólar por m2
Decidimos criar uma coluna para armazenar o preço em dólar por m2. Fizemos isso porque entendemos que 3 variáveis influenciam o preço, property_type, place_name e total_m2. Criando a coluna price_m2 entendemos que teremos somente 2 variáveis influenciando o price_m2, property_type e place_name. Temos todos os valores preenchidos na coluna divisora total_m2.

In [ ]:
df_prop['price_m2'] = df_prop.apply(lambda x: x['price_usd'] / x['total_m2'], axis=1)
df_prop.head(5)

# Verificando se foi preenchido price_m2 com valor menor ou igual a zero

In [ ]:
mask_price_m2_menor_igual_zero = df_prop['price_m2']<= 0
data_price_m2_menor_igual_zero = df_prop.loc[mask_price_m2_menor_igual_zero]
data_price_m2_menor_igual_zero

# Observando o DataFrame
Podemos perceber pelos valores máximos e mínimos das colunas total_m2, price_usd e price_m2 que temos uma variação muito grande de valores. Possivelmente temos outliers nessas colunas.

In [ ]:
df_prop.describe()

# Verificando os 50 registros com maior price_m2
Ordenamos os registros por price_m2 do maior para o menor e verificamos os registros. Aparentemente o registro de index 3901 é um outlier pois está muito distante do segundo maior. Observamos que existem imóveis com total_m2=10, aparentemente esses são valores errados e iremos eliminar do dataframe. Se olharmos a relação abaixo concluíremos que os tipos de imóveis mais caros são apartamentos e se concentram nos bairros de Boedo, Flores, San Cristobal, Almagro. O bairro Palermo possui dois imóveis entre os 5 valores mais caros por m2 porém os dois imóveis estão com 10m2 cada um e iremos eliminar esses registros.

In [ ]:
df_prop.sort_values(by='price_m2', ascending=False)[['place_name','price_m2','total_m2','property_type']].head(50)

# Eliminando registro de index 3901 - outlier de price_m2 muito elevado

In [ ]:
df_prop = df_prop.drop(3901)

# Verificando imóveis menores ou iguais a 10m2
O único registro que nos gerou dúvida em eliminar é o do tipo store porque Flores é um bairro valorizado e nos pareceu que existe a possibilidade de existir uma loja de 10m2 em um bairro valorizado.

In [ ]:
mask_total_m2_menor_igual_10 = df_prop.total_m2 <= 10
df_prop.loc[mask_total_m2_menor_igual_10]

# Eliminado registros com total_m2 menor ou igual a 10
O único registro que ficará com 10m2 é o da loja.

In [ ]:
mask_total_m2_menor_igual_10 = (df_prop.total_m2 <= 10) & (df_prop.property_type != 'store')

df_prop = df_prop.drop(df_prop[mask_total_m2_menor_igual_10].index)

mask_total_m2_menor_igual_10 = df_prop.total_m2 <= 10

df_prop.loc[mask_total_m2_menor_igual_10]

# Aparentemente ainda existem muitos valores outliers na variável price_m2.

In [ ]:
sns.boxplot(x='price_m2', data=df_prop, orient='h')

# Criando nova coluna concatenando property e place para analisar em boxplot

In [ ]:
df_prop['property_place'] = df_prop['property_type'] + '_' + df_prop['place_name']

# Verificando outliers do tipo apartamento

In [ ]:
sns.catplot(y="property_place", x="price_m2",
                data=df_prop[df_prop['property_type']=='apartment'], kind="box",
                height=10, orient='h');

# Verificando outliers do tipo PH

In [ ]:
sns.catplot(y="property_place", x="price_m2",
                data=df_prop[df_prop['property_type']=='PH'], kind="box",
                height=10, orient='h');

# Verificando outliers do tipo store

In [ ]:
sns.catplot(y="property_place", x="price_m2",
                data=df_prop[df_prop['property_type']=='store'], kind="box",
                height=10, orient='h');

# Verificando outliers do tipo house

In [ ]:
sns.catplot(y="property_place", x="price_m2",
                data=df_prop[df_prop['property_type']=='house'], kind="box",
                height=10, orient='h');

# Criando novo DataFrame agrupando place_name e property_type
Usaremos esse DataFrame para correr os registros de cada bairro buscando o valor de price_m2 desde 25 % abaixo do primeiro e até 25% acima do terceiro quartil. Manteremos somente os dados nesse intervalo.

In [ ]:
df_prop2 = pd.DataFrame(df_prop, columns=['property_type','place_name','property_place','price_m2'])
df_prop2 = df_prop2.groupby(['property_type','place_name','property_place']).count().reset_index()
df_prop2

In [ ]:
df_prop3 = pd.DataFrame(df_prop)

for i in df_prop2.index:
    property_type = df_prop2.loc[i, 'property_type']
    place_name    = df_prop2.loc[i, 'place_name']
    
    Q1 = np.percentile(df_prop[(df_prop['place_name']==place_name) & (df_prop['property_type']==property_type)]['price_m2'], 25,
                       interpolation = 'midpoint') * 0.75
 
    Q3 = np.percentile(df_prop[(df_prop['place_name']==place_name) & (df_prop['property_type']==property_type)]['price_m2'], 75,
                       interpolation = 'midpoint') * 1.25
    
    mask_iq = (df_prop3['property_type']==property_type) & (df_prop3['place_name']==place_name) & ((df_prop3['price_m2']<Q1) | (df_prop3['price_m2']>Q3))
    
    #print(f"Deletando {property_type} {place_name} entre {Q1} e {Q3} {mask_iq.count()} registros")
    
    df_prop3.drop(df_prop3[mask_iq].index, inplace=True)
df_prop3

# Verificando quantos registros sobraram de cada tipo
 - No DataFrame original tínhamos 14428 apartamentos e ficamos com 12529. Eliminamos cerca de 14.2% de apartamentos;
 - No DataFrame original tínhamos 1142 PHs e ficamos com 1013. Eliminamos cerca de 12,3% de PHs;
 - No DataFrame original tínhamos 888 houses e ficamos com 743. Eliminamos cerca de 16.4% de houses;
 - No DataFrame original tínhamos 765 stores e ficamos com 611. Eliminamos cerca de 21.1% de stores;
 - No DataFrame original tínhamos 17223 registros e ficamos com 14896. Eliminamos cerca de 15.5% do DataFrame;

In [ ]:
df_prop3.value_counts('property_type')

# Verificando o resultado da eliminação de outliers do tipo apartamento

In [ ]:
sns.catplot(y="property_place", x="price_m2",
                data=df_prop3[df_prop3['property_type']=='apartment'], kind="box",
                height=10, orient='h');

# Verificando o resultado da eliminação de outliers do tipo PH

In [ ]:
sns.catplot(y="property_place", x="price_m2",
                data=df_prop3[df_prop3['property_type']=='PH'], kind="box",
                height=10, orient='h');

# Verificando o resultado da eliminação de outliers do tipo house

In [ ]:
sns.catplot(y="property_place", x="price_m2",
                data=df_prop3[df_prop3['property_type']=='house'], kind="box",
                height=10, orient='h');

# Verificando o resultado da eliminação de outliers do tipo store

In [ ]:
sns.catplot(y="property_place", x="price_m2",
                data=df_prop3[df_prop3['property_type']=='store'], kind="box",
                height=10, orient='h');

# Verificando como ficou o DataFrame

In [7]:
df_prop3.describe()

NameError: name 'df_prop3' is not defined

# Identificando place_name com menor número de imóveis e iremos eliminar os registros em bairros com menos de 10 registros
 - Primeiro iremos criar uma coluna para armazenar quantos registros existem por bairro

In [ ]:
df_prop3['count_place'] = df_prop3.groupby('place_name')['place_name'].transform('count')

 - Abaixo verificamos quantas linhas temos no DataFrame

In [ ]:
print(df_prop3.shape)

 - Verificamos se foi alimentado o contador de bairros

In [ ]:
df_prop3.loc[0:5,['place_name','count_place']].sort_values('count_place')

 - Verificando o número de registros por bairro

In [ ]:
df_prop3.place_name.value_counts()

In [ ]:
# criação de uma mascara com quantidade menor que 10 e observando a porcentagem de registros
df_prop3_mask_menor10 = df_prop3.count_place < 10
# observando a porcentagem de valores com menos de 10 propriedades por bairro
df_prop3_menor10 = df_prop3.loc[df_prop3_mask_menor10,['count_place']]
df_prop3_menor10.shape[0] / df_prop3.shape[0] * 100

In [ ]:
# dropando as linhas com a mascara criada
df_prop3 = df_prop3.drop(df_prop3[df_prop3_mask_menor10].index)

In [ ]:
# observando a quantidade de linhas após o drop 
df_prop3.shape

In [ ]:
# observando a quantidade de valores após dropar os registros
df_prop3['place_name'].value_counts().sort_values().head(5)

In [ ]:
# observando os valores da coluna price_m2
df_prop3['price_m2'].sort_values().head(10)

In [ ]:
df_prop3.head(5)

# Verificamos se há algum imóvel com total_m2 muito alto ou baixo. Poderia influenciar no price_m2

In [ ]:
df_prop3[['place_name','property_type','price_m2','total_m2']].sort_values('total_m2', ascending=False)

# Criando array com os quartis com a média do preço por metro quadrado
Percebemos a influência de alguns bairros com price_m2 mais alto no último quartil, Apartamentos em Boedo, por exemplo.

In [ ]:
# calculando os quartis dos preços por m2 na serie  
q_price_m2	 = pd.qcut(df_prop3.price_m2	, 4)
print(type(q_price_m2))
q_price_m2

# Criando variável da média do price_m2 por property_type e place_name

In [ ]:
df_prop3['avg_price_m2'] = df_prop3.groupby('property_place')['price_m2'].transform('mean')

In [ ]:
df_prop3.head(10)

# Análise espacial da localização das propriedades com o Geopandas
Verificamos o DataFrame e percebemos que existem somente registros de Buenos Aires Capital. Diante disso importamos um arquivo com os perímetros dos bairros de Buenos Aires Capita e posteriormente iremos analisar se há algum registro fora e eliminá-los.

In [ ]:
df_bairros = pd.read_csv("../Data/Neighborhoods.csv", encoding = 'latin1')
df_bairros.head()

 - Convertendo os bairros de DataFrame para GeoDataFrame

In [8]:
df_bairros["WKT"] = df_bairros["WKT"].apply(shapely.wkt.loads)
geo_bairros = gpd.GeoDataFrame(df_bairros, geometry = 'WKT' )
type(geo_bairros)

NameError: name 'df_bairros' is not defined

 - Observando os bairros geograficamente

In [ ]:
geo_bairros.plot(color = 'white', edgecolor='black', figsize=(20,12))

 - Visualização das colunas do dataframe de propriedades

In [ ]:
df_prop3.info()

 - Conversão das colunas latitude (lat) e longitude (lon) do dataframe propriedade em pontos geométricos

In [ ]:
geometria = gpd.points_from_xy(df_prop3['lon'],df_prop3['lat'])
geometria[0:2]

 - Inserindo os pontos geometricos numa nova coluna chamada geometry do dataframe de propriedade df_prop

In [ ]:
df_prop3 = gpd.GeoDataFrame(df_prop3,
                            geometry = geometria
                           )

 - Observando as colunas com os dados geometricos no dataframe de propriedades df_prop3

In [ ]:
df_prop3.loc[ : , ['geometry', 'lat', 'lon', 'property_type', 'place_name']]

 - Observando os dados de latitude e longitude para verificar se existe algum ponto muito distante

In [ ]:
df_prop3.sort_values(by='lat')

# Iremos analisar alguns gráficos com o preço médio por m2 por tipo de propriedade e bairro

In [9]:
df_apart = df_prop3.loc[ : , ['property_type', 'place_name', 'avg_price_m2']]

avg_bairro_apart = df_apart.loc[df_apart['property_type'] == 'apartment']
avg_bairro_apart['place_name'] = avg_bairro_apart['place_name'].str.upper()

geodata_apart = pd.merge(left=avg_bairro_apart, right=df_bairros, left_on='place_name', right_on='NEIGHBORHOOD', how='inner')
geodata_apart = geodata_apart.rename(columns={"WKT": "geometry"})
geodata_apart = geodata_apart.drop(columns=['NEIGHBORHOOD','COMMUNE','PERIMETER','AREA'])
geodata_apart = geodata_apart.drop_duplicates(subset=['place_name'])

geodf_apart = gpd.GeoDataFrame(geodata_apart)

ax = geodf_apart.loc[geodf_apart['property_type'] == 'apartment'].plot(
    figsize=(20,16),
    edgecolor='black',column='avg_price_m2',
    legend=True,
    legend_kwds={'label': "Valor médio por Bairro",'orientation':"vertical" })

for i in geodf_apart.index:
    plt.text(geodf_apart.geometry.centroid[i].coords[0][0], geodf_apart.geometry.centroid[i].coords[0][1], geodf_apart.place_name[i],
             fontsize=8, horizontalalignment='center', verticalalignment='center', color = 'cyan')
    
ax.set_axis_off();
ax.set_title("PREÇO MÉDIO POR M2 DE APARTAMENTOS",fontsize=26);

NameError: name 'df_prop3' is not defined

In [ ]:
df_PH = df_prop3.loc[ : , ['property_type', 'place_name', 'avg_price_m2']]

avg_bairro_PH = df_PH.loc[df_PH['property_type'] == 'PH']
avg_bairro_PH['place_name'] = avg_bairro_PH['place_name'].str.upper()

geodata_PH = pd.merge(left=avg_bairro_PH, right=df_bairros, left_on='place_name', right_on='NEIGHBORHOOD', how='inner')
geodata_PH = geodata_PH.rename(columns={"WKT": "geometry"})
geodata_PH = geodata_PH.drop(columns=['NEIGHBORHOOD','COMMUNE','PERIMETER','AREA'])
geodata_PH = geodata_PH.drop_duplicates(subset=['place_name'])

geodf_PH = gpd.GeoDataFrame(geodata_PH)

ax = geodf_PH.loc[geodf_PH['property_type'] == 'PH'].plot(
    figsize=(20,16),
    edgecolor='black',column='avg_price_m2',
    legend=True,
    legend_kwds={'label': "Valor médio por Bairro",'orientation':"vertical" })

for i in geodf_PH.index:
    plt.text(geodf_PH.geometry.centroid[i].coords[0][0], geodf_PH.geometry.centroid[i].coords[0][1], geodf_PH.place_name[i],
             fontsize=8, horizontalalignment='center', verticalalignment='center', color = 'cyan')
    
ax.set_axis_off();
ax.set_title("PREÇO MÉDIO POR M2 DE PH",fontsize=26);

In [ ]:
df_store = df_prop3.loc[ : , ['property_type', 'place_name', 'avg_price_m2']]

avg_bairro_store = df_store.loc[df_store['property_type'] == 'store']
avg_bairro_store['place_name'] = avg_bairro_store['place_name'].str.upper()

geodata_store = pd.merge(left=avg_bairro_store, right=df_bairros, left_on='place_name', right_on='NEIGHBORHOOD', how='inner')
geodata_store = geodata_store.rename(columns={"WKT": "geometry"})
geodata_store = geodata_store.drop(columns=['NEIGHBORHOOD','COMMUNE','PERIMETER','AREA'])
geodata_store = geodata_store.drop_duplicates(subset=['place_name'])

geodf_store = gpd.GeoDataFrame(geodata_store)

ax = geodf_store.loc[geodf_store['property_type'] == 'store'].plot(
    figsize=(20,16),
    edgecolor='black',column='avg_price_m2',
    legend=True,
    legend_kwds={'label': "Valor médio por Bairro",'orientation':"vertical" })

for i in geodf_store.index:
    plt.text(geodf_store.geometry.centroid[i].coords[0][0], geodf_store.geometry.centroid[i].coords[0][1], geodf_store.place_name[i],
             fontsize=8, horizontalalignment='center', verticalalignment='center', color = 'cyan')
    
ax.set_axis_off();
ax.set_title("PREÇO MÉDIO POR M2 DE LOJAS",fontsize=26);

In [ ]:
df_house = df_prop3.loc[ : , ['property_type', 'place_name', 'avg_price_m2']]

avg_bairro_house = df_house.loc[df_house['property_type'] == 'house']
avg_bairro_house['place_name'] = avg_bairro_house['place_name'].str.upper()

geodata_house = pd.merge(left=avg_bairro_house, right=df_bairros, left_on='place_name', right_on='NEIGHBORHOOD', how='inner')
geodata_house = geodata_house.rename(columns={"WKT": "geometry"})
geodata_house = geodata_house.drop(columns=['NEIGHBORHOOD','COMMUNE','PERIMETER','AREA'])
geodata_house = geodata_house.drop_duplicates(subset=['place_name'])

geodf_house = gpd.GeoDataFrame(geodata_house)

ax = geodf_house.loc[geodf_house['property_type'] == 'house'].plot(
    figsize=(20,16),
    edgecolor='black',column='avg_price_m2',
    legend=True,
    legend_kwds={'label': "Valor médio por Bairro",'orientation':"vertical" })

for i in geodf_house.index:
    plt.text(geodf_house.geometry.centroid[i].coords[0][0], geodf_house.geometry.centroid[i].coords[0][1], geodf_house.place_name[i],
             fontsize=8, horizontalalignment='center', verticalalignment='center', color = 'cyan')
    
ax.set_axis_off();
ax.set_title("PREÇO MÉDIO POR M2 DE CASAS",fontsize=26);

# Visualização geográfica das propriedades por tipos de propriedade
 - Criaçao de um dataframe para propriedade do tipo apartamento

In [ ]:
df_apartment = df_prop3.loc[df_prop3['property_type'] == 'apartment', :]
display(df_apartment.head(3))

 - Criação de um série para receber a coluna geometry de propriedades tipo apartamento

In [ ]:
geo_apartment = df_apartment['geometry']

 - Criaçao de um dataframe para propriedade do tipo HOUSE

In [ ]:
df_house = df_prop3.loc[df_prop3['property_type'] == 'house', :]
display(df_house.head(3))

 - Criação de um série para receber a coluna geometry de propriedades tipo HOUSE

In [ ]:
geo_house = df_house['geometry']

 - Criaçao de um dataframe para propriedade do tipo PH

In [ ]:
df_PH = df_prop3.loc[df_prop3['property_type'] == 'PH', :]
display(df_PH.head(3))

 - Criação de um série para receber a coluna geometry de propriedades tipo PH

In [ ]:
geo_PH = df_PH['geometry']

 - Criaçao de um dataframe para propriedade do tipo store

In [ ]:
df_store = df_prop3.loc[df_prop3['property_type'] == 'store', :]
display(df_store.head(3))

 - Criação de um série para receber a coluna geometry de propriedades tipo PH

In [ ]:
geo_store = df_store['geometry']

In [ ]:
plt.rcParams["figure.figsize"] = (20, 14)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
geo_bairros.plot(ax = ax,
                 color = 'white',
                 edgecolor = 'black',
                )
geo_house.plot(ax = ax,
               color = 'red'
              )
geo_apartment.plot(ax = ax,
                   color = 'blue',
                   alpha = 0.1
                  )
geo_PH.plot(ax = ax,
            color = 'green'
           )
geo_store.plot(ax = ax,
               color = 'purple'
              )
plt.legend(["House","Apartment","PH","Store"],
           bbox_to_anchor = (1.05, 0.6)
          )
                           
plt.show();

# Verificamos no gráfico acima que existem registros fora de Buenos Aires Capital. Em seguida iremos eliminar esses registros.

 - Quantidade de linhas e colunas

In [ ]:
df_prop3.shape

 - Fazendo um join entre o dataframe das propriedades e bairro, o df_prop3 será atualizado apenas com as propriedades cuja localização está dentro de algum poligono de bairro

In [ ]:
df_prop3 = gpd.sjoin(df_prop3, geo_bairros, how='inner', op='within')

 - Quantidade de linhas e colunas após o join

In [ ]:
df_prop3.shape

# Alimentando as máscaras novamente para plotar o gráfico e checar se ficou ainda algum registro fora de Buenos Aires Capital

In [ ]:
df_apartment = df_prop3.loc[df_prop3['property_type'] == 'apartment', :]
geo_apartment = df_apartment['geometry']
df_house = df_prop3.loc[df_prop3['property_type'] == 'house', :]
geo_house = df_house['geometry']
df_PH = df_prop3.loc[df_prop3['property_type'] == 'PH', :]
geo_PH = df_PH['geometry']
df_store = df_prop3.loc[df_prop3['property_type'] == 'store', :]
geo_PH = df_PH['geometry']
geo_store = df_store['geometry']

In [ ]:
plt.rcParams["figure.figsize"] = (20, 14)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
geo_bairros.plot(ax = ax,
                 color = 'white',
                 edgecolor = 'black',
                )
geo_house.plot(ax = ax,
               color = 'red'
              )
geo_apartment.plot(ax = ax,
                   color = 'blue',
                   alpha = 0.1
                  )
geo_PH.plot(ax = ax,
            color = 'green'
           )
geo_store.plot(ax = ax,
               color = 'purple'
              )
plt.legend(["House","Apartment","PH","Store"],
           bbox_to_anchor = (1.05, 0.6)
          )
                           
plt.show();

# Análise de Correlação

- Gráfico de dispersão do preço das propriedades tipo apartamento e o tamanho em m2

In [10]:
sns.set_style('whitegrid')
plt.figure(figsize = (20, 12))
sns.scatterplot(data = df_prop3[df_prop3['property_type']=='apartment'], x = "price_usd", y = "total_m2")
plt.xlabel("Price USD"); plt.ylabel("Total M2");plt.title("Scatter Plot")
plt.show()

NameError: name 'df_prop3' is not defined

<Figure size 1440x864 with 0 Axes>

# Abaixo verificamos a influência dos bairros nos preços por m2 em cada tipo de propriedade

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize = (20, 15))
sns.scatterplot(data = df_prop3[df_prop3['property_type']=='apartment'], x = "price_m2", y = "place_name")
plt.xlabel("Price M2"); plt.ylabel("Place Name");plt.title("Scatter Plot")
plt.show()

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize = (20, 12))
sns.scatterplot(data = df_prop3[df_prop3['property_type']=='house'], x = "price_m2", y = "place_name")
plt.xlabel("Price M2"); plt.ylabel("Place Name");plt.title("Scatter Plot")
plt.show()

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize = (20, 12))
sns.scatterplot(data = df_prop3[df_prop3['property_type']=='PH'], x = "price_m2", y = "place_name")
plt.xlabel("Price M2"); plt.ylabel("Place Name");plt.title("Scatter Plot")
plt.show()

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize = (20, 12))
sns.scatterplot(data = df_prop3[df_prop3['property_type']=='store'], x = "price_m2", y = "place_name")
plt.xlabel("Price M2"); plt.ylabel("Place Name");plt.title("Scatter Plot")
plt.show()

- Gráfico de dispersão do preço das propriedades tipo apartamento nos bairros

In [ ]:
sns.pairplot(data = df_prop3[['property_type','place_name','lat','lon','total_m2','price_usd','price_m2','avg_price_m2']], hue = 'property_type', height = 1.8)

# Após a análise dos gráficos de correlação percebemos que há uma relação forte entre as colunas price_usd e total_m2.
 - Durante nossa etapa de limpeza e transformação optamos por criar uma coluna price_m2 que é a divisão entre o price_usd e o total_m2;
 - Sendo assim para nosso modelo de dados deixaremos as colunas float de price_m2 e avg_price_m2. A avg_price_m2 é uma média dos preços por property_type e place_name. Entendemos que ela possa servir como um indicador para auxiliar o machine learning a entender a variação de preços por property_type e place_name;
 - Criaremos abaixo as variáveis dummies por property_type e place_name.

# Dummy da variável tipo de propriedade 

- Criação de um novo df_prop5 com as coluna property_place e o preço m2

In [ ]:
df_prop3.head(3)

In [ ]:
df_prop5 = df_prop3.loc[:,["property_type","place_name","price_m2","avg_price_m2"]]

- verificando as primeiras linhas e quantidade registros

In [ ]:
df_prop5.head(3)

In [ ]:
df_prop5.shape[0]

- Criação Dummy Indicator columns para criar colunas fictícias para property_type, utilizamos drop_first para eliminar a primeira variável

In [ ]:
df_prop5_dummy = pd.get_dummies(df_prop5, columns=['property_type'], prefix="type", prefix_sep="_", drop_first=False)

In [ ]:
df_prop5_dummy.head(5)

In [ ]:
df_prop5_dummy = pd.get_dummies(df_prop5_dummy, columns=['place_name'], prefix="place", prefix_sep="_", drop_first=False)

In [11]:
df_prop5_dummy.head(5)

NameError: name 'df_prop5_dummy' is not defined

In [ ]:
df_prop5_dummy.shape

# AQUI COMEÇA 

# Inclusão coluna com a distancia das propriedades até o obelisco

# Definindo as coordenadas do Obelisco

In [ ]:
from shapely.geometry import Point
ponto_obelisco_p = Point(-58.381555, -34.605425)

In [ ]:
type(ponto_obelisco_p)

In [ ]:
ponto_obelisco = (-58.381555, -34.605425)

In [ ]:
type(ponto_obelisco)

Criação da coluna com a distancia até o obelisco no df_prop3

In [ ]:
import geopy.distance

for i in df_prop3.index:
    df_prop3.loc[i, 'dist_obelisk'] = geopy.distance.geodesic(ponto_obelisco,
                                                                  (df_prop3.loc[i, 'geometry'].x ,df_prop3.loc[i, 'geometry'].y)
                                                                 ).meters

Observando as primeiras linhas do df_prop3

In [ ]:
df_prop3.loc[:4, ['dist_obelisk', 'geometry', 'lat', 'lon','property_type', 'place_name']]

Vizualização do pairplot com  nova coluna para ver se há correlação com as demais variáveis 

In [ ]:
sns.pairplot(data = df_prop3[['property_type','total_m2','price_usd','price_m2','avg_price_m2','dist_obelisk']], hue = 'property_type');

Heatmap correlação

In [ ]:
df_prop3_corr = df_prop3.loc[:,['place_name','total_m2','price_usd','price_m2','avg_price_m2','geometry','dist_obelisk']]
df_prop3_corr.head(5)

In [ ]:
sns.heatmap(df_prop3_corr.corr(), 
            annot = True, 
            vmin = -1, 
            cmap = 'Blues'
           );

Observando se há padrões relevantes a olho nu entre a distancia e o preço m2 com scatterplot

In [12]:
df_prop3_corr = df_prop3.loc[:,['property_type','dist_obelisk','price_m2']]

NameError: name 'df_prop3' is not defined

In [ ]:
df_prop3_corr.describe().T

# INICIO PROJETO PARTE II

# Segmentação dos dados para trabalhar com cerca de 2.000 registros

Observando a quantidade de propriedades por bairro

In [ ]:
df_prop3.value_counts('place_name').head(10)

Segmentando o conjunto de dados por tipo de propriedade apartamento e features que serão utilizadas

In [ ]:
df_prop4 = df_prop3.loc[:,['place_name','property_type','price_m2','avg_price_m2','total_m2','price_usd','dist_obelisk']]

Quantidade de registros do segmento de dados df_prop4

In [ ]:
print(df_prop4.shape)

Observando a quantidade de propriedades por bairro

In [ ]:
df_prop4.value_counts('place_name', ascending=False).head(10)

Observando a quantidade de linhas e colunas do dataframe

In [ ]:
df_prop4.shape

Criação de uma nova coluna com a quantidade de propriedades por bairro

In [ ]:
df_prop4['count_place2'] = df_prop4.groupby('place_name')['place_name'].transform('count')

In [ ]:
print('Quantidade de linhas e colunas: ', df_prop4.shape)

Observando quais os bairros do dataframe

In [13]:
df_prop4.value_counts('place_name').head(15)

NameError: name 'df_prop4' is not defined

Criaçãode uma máscara para segmentar por bairros com mais de 200 propriedades

In [ ]:
mask_count_place2 = df_prop4.count_place2 >= 900
df_prop4 = df_prop4.loc[mask_count_place2]

In [ ]:
df_prop4.value_counts('place_name')

In [ ]:
print('Quantidade de linhas e colunas: ', df_prop4.shape)

Observando a correlação entre as features

In [ ]:
sns.heatmap(df_prop4.corr(), 
            vmin = -1, 
            vmax = 1, 
            center = 0, 
            cmap = "YlGnBu"
           );

In [ ]:
sns.set(style = "darkgrid")
sns.scatterplot(x = "total_m2", y = "price_m2", hue = df_prop4.place_name.tolist(), data = df_prop4)
plt.show()

# Normalização dos dados

In [ ]:
print(df_prop4.shape)
df_prop4.head(5)

Normalização das colunas price_m2

In [ ]:
xs = df_prop4["total_m2"]
ys = df_prop4["price_usd"]

plt.scatter(xs, ys)
plt.xlabel("total_m2")
plt.ylabel("price_m2")
plt.show()

Observando os dados normalizados com o scikit-learn

In [ ]:
from sklearn import preprocessing

xs = preprocessing.scale(df_prop4["total_m2"])
ys = preprocessing.scale(df_prop4["price_usd"])

plt.scatter(xs, ys, color = 'r')
plt.xlabel("total_m2 standardized")
plt.ylabel("price_m2 standardized")
plt.show()

In [ ]:
plt.figure(figsize = (12, 4))
ax1 = plt.subplot(121)
ax1.set_title("Variable escalada Min-Max")
ax1.hist(ys, bins = 30);

ax2 = plt.subplot(122)
ax2.set_title("Variable sin reescalar")
ax2.hist(df_prop4["price_usd"], bins = 30);

# Criação de um modelo de clustering não supervisionado a fim de  descobrir padrões ou estruturas subjacentes nas informações que nos permitam ter insights relevantes entre o total m2 e o preço m2

In [14]:
df_prop4_cluster1 = df_prop4.loc[:,['price_usd','total_m2']]

NameError: name 'df_prop4' is not defined

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters = 4)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = df_prop4_cluster1
X_scaled = scaler.fit_transform(X) # X_scaled são os dados transformados

In [ ]:
kmeans.fit(X_scaled)

In [ ]:
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [ ]:
plt.figure(figsize = (8,5))
sns.scatterplot(x = X_scaled[ : , 0 ], y = X_scaled[ : , 1 ], hue = labels, legend = 'full');

plt.xlabel('total_m2',fontsize = 15); plt.ylabel('price_m2', fontsize = 15);
plt.scatter(centroids[ : , 0 ], centroids[ : , 1 ], marker = 'x', s = 50, color = 'r');

# MODELO 1: REGRESSÃO LINEAR SIMPLES

Feature: total_m2
Target: price_m2

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 16

In [ ]:
df_prop4.head(5)

In [ ]:
type(df_prop4)

Passando o modelo de geodataframe para dataframe

In [ ]:
df_prop4 = pd.DataFrame(df_prop4)

In [ ]:
type(df_prop4)

Observando a correlação entre price_m2 e total_m2

In [ ]:
df_prop4.plot(kind = 'scatter', 
           x = 'total_m2', 
           y = 'price_usd', 
           alpha = 0.2
          );

In [ ]:
sns.lmplot(x = 'total_m2', 
           y = 'price_usd', 
           data = df_prop4, 
           aspect = 1.45,
           scatter_kws = {'alpha': 0.2}
          );

# NOVA Construindo um modelo de Regressão Linear Simples com price_usd e total_m2

In [15]:
from sklearn.linear_model import LinearRegression

In [16]:
model = LinearRegression(fit_intercept = True)
model

LinearRegression()

Criação de uma matriz de `features` e o vetor `target`, que usaremos para ajustar nosso modelo de regressão linear simples.

In [17]:
# Criamos X e y
feature_cols = ['total_m2']
X = df_prop4[feature_cols]
y = df_prop4.price_usd

# Checamos a forma e o tipo de cada um
print("Shape X:", X.shape)
print("Type X:", type(X))
print("Shape y:", y.shape)
print("Type y:", type(y))

NameError: name 'df_prop4' is not defined

Separando Conjunto de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state = 1)

Ajuste do modelo aos dados de treino

In [ ]:
model.fit(Xtrain, ytrain)

verificando os valores de que o modelo aprendeu inspecionando os atributos `intercept_` e` coef_`, respectivamente:

In [ ]:
model.coef_

In [ ]:
model.intercept_

Prever rótulos para dados desconhecidos usando o metodo predict

In [ ]:
ypred = model.predict(Xtest)
ypred

Observando as métricas:

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print ('MAE:', mean_absolute_error(ytest, ypred).round(2))
print ('MSE:', mean_squared_error(ytest, ypred).round(2))
print ('RMSE:', np.sqrt(mean_squared_error(ytest, ypred)).round(2))
print ('R2:', r2_score(ytest, ypred).round(2))

Observações: As funções de perda poderiam ser minimizadas estão consideravelmente altas e o R2 está distante de 1

# Modelo de Regressão Linear Simples com price_m2 x Bairros


# Adicionando novas features com dummy

Removendo a coluna count_place2

In [ ]:
df_prop4.head(2)

In [ ]:
df_prop4 = df_prop4.loc[:,['place_name','property_type','price_m2','avg_price_m2','total_m2','price_usd','dist_obelisk']]

Adicionando a a coluna dummy para tipo de propriedade

In [ ]:
df_prop4_dummy = pd.get_dummies(df_prop4, columns=['property_type'], prefix="type", prefix_sep="_", drop_first=True)

In [ ]:
df_prop4_dummy.head(5)

Adicionando a a coluna dummy para place_name

In [ ]:
df_prop4_dummy = pd.get_dummies(df_prop4_dummy, columns=['place_name'], prefix="place", prefix_sep="_", drop_first=True)

In [ ]:
df_prop4_dummy.head(5)

In [ ]:
df_prop4_dummy.shape

In [ ]:
df_prop4_dummy.head(5)

Criamos uma nova lista de features com as variáveis dummy

In [ ]:
sns.heatmap(df_prop4_dummy.corr(), 
            vmin = -1, 
            vmax = 1, 
            center = 0, 
            cmap = "YlGnBu"
           );

In [ ]:
df_prop4_dummy.head(2)

In [ ]:
feature_cols = ['avg_price_m2','total_m2','price_m2','dist_obelisk','type_apartment','type_house','type_store','type_Caballito','type_Palermo']

In [ ]:
# criamos novamente X and y
X = df_prop4_dummy[feature_cols]
y = df_prop4_dummy.price_usd

# criamos o modelo e o ajustamos aos dados
linreg = LinearRegression()
linreg.fit(X, y)

# Imprimimos os coeficientes
print (linreg.intercept_)
print (linreg.coef_)

Para observá-lo melhor olhamos o nome com o coeficiente

In [ ]:
list(zip(feature_cols, linreg.coef_))

Separando Conjunto de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state = 1)

Ajuste do modelo aos dados de treino

In [ ]:
model.fit(Xtrain, ytrain)

In [ ]:
print('model:', model.score(X_all_test, y_all_test))

verificando os valores de que o modelo aprendeu inspecionando os atributos `intercept_` e` coef_`, respectivamente:

In [ ]:
model.coef_

In [ ]:
model.intercept_

Prever rótulos para dados desconhecidos usando o metodo predict

In [ ]:
ypred = model.predict(Xtest)
ypred

Observando as métricas:

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print ("y_test sample: ", ytest.values[0: 10])
print ("y_pred sample: ", ypred[0: 10].astype(int))
print ('MAE:', mean_absolute_error(ytest, ypred).round(2))
print ('MSE:', mean_squared_error(ytest, ypred).round(2))
print ('RMSE:', np.sqrt(mean_squared_error(ytest, ypred)).round(2))
print ('R2:', r2_score(ytest, ypred).round(2))

Função para verificar os valores absolutos

In [ ]:
def mape(y_test, pred):
    y_test, pred = np.array(y_test), np.array(pred)
    mape = np.mean(np.abs((y_test - pred) / y_test))
    return mape

In [ ]:
print('MAPE:', mape(ytest, ypred))

Observamos acima que com as variáveis dummys o R2 subiu e os valores dos resíduos minimizaram

## Criação da função definida train_test_error() 
# a função aceita uma lista de recursos, faz a divisão entre treino e teste, reservando 25% das observações para teste e retorna o teste RMSE.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def train_test_error(feature_cols):
    X = df_prop4_dummy[feature_cols]
    y = df_prop4_dummy.price_usd

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    y_testm, predm = np.array(y_test), np.array(y_pred)
    mape = np.mean(np.abs((y_testm - predm) / y_testm))
  
    
    print (feature_cols)
    print ("y_test sample: ", y_test.values[0: 10])
    print ("y_pred sample: ", y_pred[0: 10].astype(int))
    print ('MAE:', mean_absolute_error(y_test, y_pred))
    print ('MSE:', mean_squared_error(y_test, y_pred))
    print ('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
    print ('R2:', r2_score(y_test, y_pred))
    print('MAPE:', mape)
    print ()
    return 

In [ ]:
train_test_error(['avg_price_m2','total_m2','price_m2','dist_obelisk','type_apartment','type_house','type_store','type_Caballito','type_Palermo'])

In [ ]:
train_test_error(['avg_price_m2','total_m2','type_apartment','type_house','type_store','type_Caballito','type_Palermo'])

In [ ]:
train_test_error(['total_m2','type_apartment','type_house','type_store','type_Caballito','type_Palermo'])

# PREMISSAS DE GAUSS MARKOUS.

In [ ]:
X = df_prop4_dummy[['avg_price_m2','total_m2','price_m2','dist_obelisk','type_apartment','type_house','type_store','type_Caballito','type_Palermo']]
y = pd.Series(df_prop4_dummy.price_usd, 
              name = 'price_usd'
             )


In [ ]:
import statsmodels.api as sm

X_constant = sm.add_constant(X)
lin_reg = sm.OLS(y,X_constant).fit()
lin_reg.summary()

Verificando a linearidade do modelo

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format ='retina'
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

def linearity_test(model, y):
    '''
    função para visualizar e identificar suposições de linearidade sobre regressão linear
    
    Args:
    * model - fitted OLS model from statsmodels
    * y - observed values
    '''
    fitted_vals = model.predict()
    resids = model.resid

    fig, ax = plt.subplots(1,2)
    
    sns.regplot(x = fitted_vals, 
                y = y, 
                lowess = True, 
                ax = ax[0], 
                line_kws = {'color': 'red'}
               )
    ax[0].set_title('Observados vs. Valores Previstos', 
                    fontsize = 16
                   )
    ax[0].set(xlabel = 'Previstos', 
              ylabel = 'Observados')

    sns.regplot(x = fitted_vals, 
                y = resids, 
                lowess = True, 
                ax = ax[1], 
                line_kws = {'color': 'red'}
               )
    ax[1].set_title('Residuos vs. Valores Previstos', 
                    fontsize = 16
                   )
    ax[1].set(xlabel = 'Previstos', 
              ylabel = 'Residuos'
             )
    
linearity_test(lin_reg, y)    

Para detectar a não linearidade, você pode inspecionar gráficos de valores observados versus valores previstos ou residuais versus valores previstos. O resultado desejado é que os pontos sejam distribuídos simetricamente em torno de uma linha diagonal no primeiro gráfico ou em torno de uma linha horizontal no último. Em ambos os casos, com uma variação aproximadamente constante.

A observação de um padrão inclinado indica que o modelo comete erros sistemáticos cada vez que faz previsões incomumente grandes ou pequenas. Quando o modelo contém muitos recursos, a não linearidade também pode ser revelada por padrões sistemáticos nos gráficos dos resíduos em comparação com os recursos individuais.

Observamos acima que a suposição de linearidade não é cumprida, 

A expectativa ou média dos resíduos igual a zero

In [ ]:
lin_reg.resid.mean()

Verificando a colinearidade entre as variáveis

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = [variance_inflation_factor(X_constant.values, i) for i in range(X_constant.shape[1])]
pd.DataFrame({'vif': vif[1:]}, index=X.columns).T

Para lidar com a multicolinearidade, precisamos remover iterativamente os recursos com valores altos de **VIF**. 
Uma regra geral para eliminação pode ser VIF maior que 10 (5 também é comum). 
Outra solução possível é usar o PCA para reduzir os recursos a um conjunto menor de componentes não correlacionados.
Dica: também podemos visualizar a matriz de correlação de recursos para identificar dependências entre recursos.

### Homocedasticidade (variância igual) dos resíduos
Quando os resíduos não possuem variância constante (exibem heterocedasticidade), é difícil determinar o verdadeiro desvio padrão dos erros de previsão, geralmente resultando em intervalos de confiança muito amplos/estreitos. 
Por exemplo, se a variância dos resíduos aumentar ao longo do tempo, os intervalos de confiança para as previsões fora da amostra não serão realistas. 
Outro efeito da heterocedasticidade também pode ser colocar muito peso em um subconjunto de dados ao estimar os coeficientes, o subconjunto no qual a variância do erro foi maior.
Para investigar se os resíduos têm variância igual, podemos olhar para um gráfico dos resíduos (ou resíduos padronizados) contra os valores previstos (ajustados) como fizemos na figura anterior ao verificar a linearidade do modelo. 
O que deve nos alarmar é o caso em que os resíduos crescem em função do valor previsto ou em função do tempo (no caso de séries temporais).
Também podemos usar dois testes estatísticos: Breusch-Pagan e Goldfeld-Quandt. 
Em ambos, a hipótese nula assume homocedasticidade, e um valor de p abaixo de um determinado nível (como 0,05) indica que devemos rejeitar o nulo em favor da heterocedasticidade.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

def homoscedasticity_test(model):
    '''
    Function for testing the homoscedasticity of residuals in a linear regression model.
    It plots residuals and standardized residuals vs. fitted values and runs Breusch-Pagan and Goldfeld-Quandt tests.
    
    Args:
    * model - fitted OLS model from statsmodels
    '''
    import numpy as np
    fitted_vals = model.predict()
    resids = model.resid
    resids_standardized = model.get_influence().resid_studentized_internal

    fig, ax = plt.subplots(1,2)

    sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title('Residuals vs Fitted', fontsize=16)
    ax[0].set(xlabel='Fitted Values', ylabel='Residuals')

    sns.regplot(x=fitted_vals, y=np.sqrt(np.abs(resids_standardized)), lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title('Scale-Location', fontsize=16)
    ax[1].set(xlabel='Fitted Values', ylabel='sqrt(abs(Residuals))')

    bp_test = pd.DataFrame(sms.het_breuschpagan(resids, model.model.exog), 
                           columns=['value'],
                           index=['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value'])

    gq_test = pd.DataFrame(sms.het_goldfeldquandt(resids, model.model.exog)[:-1],
                           columns=['value'],
                           index=['F statistic', 'p-value'])

    print('\n Breusch-Pagan test ----')
    print(bp_test)
    print('\n Goldfeld-Quandt test ----')
    print(gq_test)
    print('\n Residuals plots ----')

homoscedasticity_test(lin_reg)

O gráfico mostra que este modelo não atende a suposição de homoscedancy.
para contornar esse inconveniente, você pode aplicar alguma função logarítmica nas variáveis ​​preditoras ou na variável alvo.

Autocorrelação

In [ ]:
import statsmodels.tsa.api as smt

acf = smt.graphics.plot_acf(lin_reg.resid, lags=40 , alpha=0.05)
#acf.show();

Normalidade dos resíduos

In [ ]:
from scipy import stats

def normality_of_residuals_test(model):
    '''
    Function for drawing the normal QQ-plot of the residuals and running 4 statistical tests to 
    investigate the normality of residuals.
    
    Arg:
    * model - fitted OLS models from statsmodels
    '''
    sm.ProbPlot(model.resid).qqplot(line='s');
    plt.title('Q-Q plot');

    jb = stats.jarque_bera(model.resid)
    sw = stats.shapiro(model.resid)
    ad = stats.anderson(model.resid, dist='norm')
    ks = stats.kstest(model.resid, 'norm')
    
    print(f'Jarque-Bera test ---- statistic: {jb[0]:.4f}, p-value: {jb[1]}')
    print(f'Shapiro-Wilk test ---- statistic: {sw[0]:.4f}, p-value: {sw[1]:.4f}')
    print(f'Kolmogorov-Smirnov test ---- statistic: {ks.statistic:.4f}, p-value: {ks.pvalue:.4f}')
    print(f'Anderson-Darling test ---- statistic: {ad.statistic:.4f}, 5% critical value: {ad.critical_values[2]:.4f}')
    print('If the returned AD statistic is larger than the critical value, then for the 5% significance level, the null hypothesis that the data come from the Normal distribution should be rejected. ')
    
normality_of_residuals_test(lin_reg)

Dos resultados acima referidos, podemos inferir que os resíduos não seguem a distribuição gaussiana a partir da forma da parcela QQ, bem como rejeitar a hipótese nula em todos os testes estatísticos. 

#### Possíveis soluções:
* transformação não linear da variável/característica alvo
* remover/tratar possíveis aberrantes
* Pode acontecer que existam dois ou mais subconjuntos de dados com propriedades estatísticas diferentes, caso em que poderiam ser considerados como modelos separados.

## REGULARIZAÇÃO

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
feature_cols = ['avg_price_m2','total_m2','price_m2','dist_obelisk','type_apartment','type_house','type_store','type_Caballito','type_Palermo']

In [ ]:
X = df_prop4_dummy[feature_cols]
y = df_prop4_dummy.price_usd

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state = 1)

# Modelo Regresão Linear

In [ ]:
model = LinearRegression(fit_intercept = True)
model_1 = model.fit(Xtrain, ytrain)
print('Score model_1:', model_1.score(Xtest, ytest))

# Regularização modelo com Ridge CV

In [ ]:
lm_ridge = linear_model.RidgeCV(alphas = [0.1, 1, 10], 
                                normalize = True
                               ) 

model_2 = lm_ridge.fit(Xtrain, ytrain)

print('Score model_2:', model_2.score(Xtest, ytest))

In [ ]:
lm_lasso = linear_model.LassoCV(alphas = [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01,
                                          0.05, 0.1, 1, 5, 10, 15, 25],
                                normalize = True, 
                                cv = 10
                               )

model_3 = lm_lasso.fit(Xtrain, ytrain)

print('Score model_3:', model_3.score(Xtest, ytest))